# TP4 - Non-negative Matrix Factorization
The goal is to study the use of nonnegative matrix factorisation (NMF) for topic extraction from a dataset of text documents. The rationale is to interpret each extracted NMF component as being associated with a specific topic. 

Study and test the following script (introduced  on [scikit](http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html))

1. Test and comment on the effect of varying the initialisation, especially using random
nonnegative values as initial guesses (for W and H coefficients, using the notations introduced
during the lecture).
2. Compare and comment on the difference between the results obtained with `2 cost compared
to the generalised Kullback-Liebler cost.
3. Test and comment on the results obtained using a simpler term-frequency representation
as input (as opposed to the TF-IDF representation considered in the code above) when
considering the Kullback-Liebler cost.

In [39]:
###### CUSTOM NMF IMPLEMENTATION ######
# Multiplicative Update Rules for NMF #
# estimation with beta divergences    #
import numpy

# TODO: translate slides 59 [beta-divergence] & 47 [error and special cases]

def custom_NMF(V, K, W=None, H=None, steps=5000, alpha=0.0002, beta=0.02, toll=0.001):
    
    F = len(V) #Number of V rows
    N = len(V[0]) #Number of V columns

    if W is None:
        W = numpy.random.rand(F,K)
        
    if H is None:
        H = numpy.random.rand(K,N)
        
    if N != len(H[0]):
        raise ValueError("Size for H[0] is different - found "+str(len(H[0]))+" in place of "+str(N))
    if F != len(W):
        raise ValueError("Size for F is different - found "+str(len(F))+" in place of "+str(N))
    
    for step in range(steps):
#         for i in range(len(V)):
#             for j in range(len(V[i])):
#                 if V[i][j] > 0:
#                     eij = V[i][j] - numpy.dot(W[i,:],H[:,j])
                    #for k in range(K):
                        # Translate here slide 59
                        # H[k][j] = H[k][j] + alpha * (2 * eij * W[i][k] - beta * H[k][j])
                        # W[i][k] = W[i][k] + alpha * (2 * eij * H[k][j] - beta * W[i][k])
    
        # Tests with whole matrix : multiply = O | dot = *
        upd_UP = numpy.dot(W.T, numpy.multiply(pow(numpy.dot(W,H),beta-2), V))
        upd_DOWN = numpy.dot(W.T, pow(numpy.dot(W,H),beta-1))
        upd = upd_UP / upd_DOWN
        H = numpy.multiply(H, upd)
        
        upd_UP = numpy.dot(numpy.multiply(pow(numpy.dot(W,H),beta-2), V),H.T)
        upd_DOWN = numpy.dot(pow(numpy.dot(W,H),beta-1), H.T)
        upd = upd_UP / upd_DOWN
        W = numpy.multiply(W, upd)
        
        eV = numpy.dot(W,H)
        e = 0
        # Translate here slide 47 - e --> beta
        for i in range(len(V)):
            for j in range(len(V[i])):
                if V[i][j] > 0:
                    e = e + pow(V[i][j] - numpy.dot(W[i,:],H[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(W[i][k],2) + pow(H[k][j],2) )
        if e < toll:
            break
    return W, H

#######

if __name__ == "__main__":
    V = [
         [5,3,0,1],
         [4,0,0,1],
         [1,1,0,5],
         [1,0,0,4],
         [0,1,5,4],
        ]

    V = numpy.array(V) #F x N Source matrix
    K = 2

    W, H = custom_NMF(V, K)

/home/dgallitelli/anaconda3/envs/SD-TSIA214/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: overflow encountered in power
/home/dgallitelli/anaconda3/envs/SD-TSIA214/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in multiply


In [ ]:
##### TEST WITH SCIKIT-LEARN DATASET #####